In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some fixed parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 2, 3])              # Boundary condition function coefficients
dim_z = 30                            # Latent space dimension
dtype = torch.float                  # Tensor data type

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def essBoundaryFun(x):
    return 0.0
mesh.setEssentialBoundary(origin, essBoundaryFun)

def domainBoundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.setNaturalBoundary(domainBoundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
#Spepify right hand side and stiffness matrix
#Define boundary flux field
rhs = PoissonFEM.RightHandSide(mesh)
rhs.setNaturalRHS(mesh, flux)
funSpace = PoissonFEM.FunctionSpace(mesh)
K = PoissonFEM.StiffnessMatrix(mesh, funSpace)
rhs.setRhsStencil(mesh, K)

In [5]:
# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
precond = ksp.getPC()
precond.setType('cholesky')
ksp.setFromOptions() #???

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs, ksp)

In [7]:
trainingData = dta.StokesData(range(4096))
trainingData.readData(['IMG'])
# trainingData.plotMicrostruct(1)
trainingData.reshapeInputImg()

In [8]:
model = gs.GenerativeSurrogate(rom, trainingData, dim_z)

In [9]:
steps = int(50)
for s in range(steps):
    print('step = ', s)
    batchSamples = torch.LongTensor(model.batchSizeN).random_(0, trainingData.nSamples)
    model.pfStep(batchSamples)

step =  0
loss =  tensor(11750302., grad_fn=<SubBackward0>)
step =  1
loss =  tensor(11209172., grad_fn=<SubBackward0>)
step =  2
loss =  tensor(11060626., grad_fn=<SubBackward0>)
step =  3
loss =  tensor(10982057., grad_fn=<SubBackward0>)
step =  4
loss =  tensor(10942635., grad_fn=<SubBackward0>)
step =  5
loss =  tensor(10887964., grad_fn=<SubBackward0>)
step =  6
loss =  tensor(10873980., grad_fn=<SubBackward0>)
step =  7
loss =  tensor(10837051., grad_fn=<SubBackward0>)
step =  8
loss =  tensor(10788564., grad_fn=<SubBackward0>)
step =  9
loss =  tensor(10775338., grad_fn=<SubBackward0>)
step =  10
loss =  tensor(10754140., grad_fn=<SubBackward0>)
step =  11
loss =  tensor(10753859., grad_fn=<SubBackward0>)
step =  12
loss =  tensor(10739063., grad_fn=<SubBackward0>)
step =  13
loss =  tensor(10715000., grad_fn=<SubBackward0>)
step =  14
loss =  tensor(10706018., grad_fn=<SubBackward0>)
step =  15
loss =  tensor(10739767., grad_fn=<SubBackward0>)
step =  16
loss =  tensor(10731711

In [10]:
model.writer.close()
model.plotInputReconstruction()

In [11]:
f = plt.gcf()
f.suptitle('Untrained, N = 1184', fontsize=32, y=.7)

Text(0.5, 0.7, 'Untrained, N = 1184')

In [22]:
x = torch.distributions.multivariate_normal.MultivariateNormal(torch.tensor([1.0, 2.0]), torch.eye(2))

In [26]:
y = x.rsample((3, 3))

In [27]:
y.shape

torch.Size([3, 3, 2])